In [1]:
from bs4 import BeautifulSoup
import urllib.request
from selenium import webdriver
import time
import datetime # 크롤링한 시각
import pandas as pd
import re

# Melon 24Hits sampling song info & lyrics by page (2020-11-20 written)

#### I want Melon don't renew their structure for a long long time...

In [2]:
wd = webdriver.Chrome('C:/chromedriver.exe')
wd.get('https://www.melon.com/chart/index.htm')

html = wd.page_source
soup = BeautifulSoup(html, 'html.parser')
time.sleep(3)

# 정규표현식으로 Top100의 SongNumber 추출
first_sample = re.compile('tr data-song-no="(\d+)"')
tmp = first_sample.findall(str(soup))

second_sample = re.compile('\d+')
sNum = second_sample.findall(str(tmp)) # 결국 for문의 idx가 됨
time.sleep(3)
wd.close()

name = '' # 곡명
at = '' # 아티스트
alb = '' # 앨범
dt = '' # 발매일
gr = '' # 장르
flac = '' # 파일타입
likes = '' # 좋아요수

# 각각 리스트로 저장
nmList = []
atList = []
albList = []
dtList = []
grList = []
flacList = []
lkList = []

for i in range(len(sNum)): # sNum
    """ 페이지 열기 """
    if i == 0: # 반복열기 방지구문
        wd = webdriver.Chrome('C:/chromedriver.exe')
    wd.get('https://www.melon.com/song/detail.htm?songId=' + sNum[i])
    
    """ 파싱하기 """
    html_dtl = wd.page_source
    soup_dtl = BeautifulSoup(html_dtl, 'html.parser')
    time.sleep(1)

    """ 곡명 추출 : 변수명 - name """
    nm = soup_dtl.find('div', attrs={'class', 'song_name'}).get_text()
    third_sample = re.sub('\n|\t|곡명', '', nm)
    name = third_sample

    """ 아티스트 추출 : 변수명 - at"""
    at = soup_dtl.select("#downloadfrm > div > div > div.entry > div.info > div.artist > a > span")[0].string

    """ 앨범 추출 : 변수명 - alb """
    alb = soup_dtl.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[0].string

    """ 발매일 추출 : 변수명 - dt """
    dt = soup_dtl.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[1].string

    """ 장르 추출 : 변수명 - gr """
    gr = soup_dtl.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[2].string

    """ 파일타입 추출 : 변수명 - flac """ # 이하이 홀로에 flac이 없음
    try:
        flac = soup_dtl.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[3].string
    except:
        flac = 'NA'
        None
    
    """ 좋아요 수 추출 : 변수명 - likes """
    likes = soup_dtl.find('span', attrs={'id':"d_like_count", 'class':"cnt"}).string

    """ 가사 추출(가사는 반환하지 않고, txt로만 저장) : 리스트명 - lyric """
    lr = str(soup_dtl.find('div', attrs={'class':"lyric", 'id':"d_video_summary"}))
    lr = re.sub('(<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->|\n|\t|</div>)', ' ', lr)
    lr = re.sub('<br/>', '\n', lr)
    lr = '<%s>\n' % name + lr + '\n\n -------노래 끝-------\n\n' # 곡별로 가사 구분

    # 가사를 텍스트 파일로 저장
    if i == 0:
        f = open('./lyrics.txt', 'w', encoding='utf-8')
        f.write(lr)
        f.close()

    else:
        f = open('./lyrics.txt', 'a', encoding='utf-8')
        f.write(lr)
        f.close()
        
    # 곡 정보를 csv로 저장
    nmList.append(name)
    atList.append(at)
    albList.append(alb)
    dtList.append(dt)
    grList.append(gr)
    flacList.append(flac)
    lkList.append(likes)
    
    # 안내메시지
    print("[%d] : <%s> is successfully crawled" % (i+1, name))
    time.sleep(1)
    
    # selenium 종료
    if i == len(sNum): # sNum
        wd.close()
        
with open('./lyrics.txt', 'a', encoding='utf-8') as f:
    f.write('\n')
    f.write('%s written' % str(datetime.datetime.now()))

print("\n<%s>" % str(datetime.datetime.now()) + ", Crawling is done...")

# 곡 정보를 csv로 저장
songInfo = {'name':nmList, 'artist':atList, 'album':albList, 'release':dtList, 'genre':grList, 'flac':flacList, 'likes':lkList}
songInfo = pd.DataFrame(songInfo)
songInfo.to_csv('melon.csv', encoding='utf-8-sig')
songInfo.head()

[1] : <Dynamite> is successfully crawled
[2] : <힘든 건 사랑이 아니다> is successfully crawled
[3] : <Life Goes On> is successfully crawled
[4] : <잠이 오질 않네요> is successfully crawled
[5] : <Lovesick Girls> is successfully crawled
[6] : <취기를 빌려 (취향저격 그녀 X 산들)> is successfully crawled
[7] : <DON'T TOUCH ME> is successfully crawled
[8] : <Savage Love (Laxed - Siren Beat) (BTS Remix)> is successfully crawled
[9] : <오래된 노래> is successfully crawled
[10] : <딩가딩가 (Dingga)> is successfully crawled
[11] : <내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)> is successfully crawled
[12] : <When We Disco (Duet with 선미)> is successfully crawled
[13] : <VVS (Feat. JUSTHIS) (Prod. GroovyRoom)> is successfully crawled
[14] : <뻔한남자> is successfully crawled
[15] : <어떻게 이별까지 사랑하겠어, 널 사랑하는 거지> is successfully crawled
[16] : <I CAN’T STOP ME> is successfully crawled
[17] : <내 방을 여행하는 법> is successfully crawled
[18] : <흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야> is successfully crawled
[19] : <밤하늘의 별을(2020)> is successfully crawled
[20] : <가을밤에 든 생각> is su

,name,artist,album,release,genre,flac,likes
0,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Flac 16bit,"351,899"
1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다,2020.10.19,발라드,Flac 16/24bit,"72,580"
2,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,Flac 16bit,"117,337"
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,Flac 16/24bit,"69,478"
4,Lovesick Girls,BLACKPINK,THE ALBUM,2020.10.02,댄스,Flac 16bit,"144,589"


## 계획
- 실시간으로 반영되다보니 나중에 차트변동률을 구할 수도 있을 것 같다!